<a href="https://colab.research.google.com/github/srathina/Python/blob/master/Tensorflow_firststeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset


tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [0]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

In [10]:
california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe['median_house_value']/=1000.
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,0.0,2643.7,539.4,1429.6,501.2,3.9,0.2
std,2.0,2.1,0.0,2179.9,421.5,1147.9,384.5,1.9,0.1
min,-124.3,32.5,0.0,2.0,1.0,3.0,1.0,0.5,0.0
25%,-121.8,33.9,0.0,1462.0,297.0,790.0,282.0,2.6,0.1
50%,-118.5,34.2,0.0,2127.0,434.0,1167.0,409.0,3.5,0.2
75%,-118.0,37.7,0.0,3151.2,648.2,1721.0,605.2,4.8,0.3
max,-114.3,42.0,0.1,37937.0,6445.0,35682.0,6082.0,15.0,0.5
